In [1]:
import numpy as np
from models.linear import LinearRegression
from models.logistic import LogisticRegression
from metrics.classification_metrics import *

In [2]:
n_samples = 500
np.random.seed(42)

classes = ["A", "B", "C", "D"]

random_classes = np.random.choice(classes, size=n_samples)


class_to_int = {label: idx for idx, label in enumerate(classes)}
random_classes_int = np.vectorize(class_to_int.get)(random_classes)

In [3]:
X = np.random.normal(2, 2, n_samples)
Z = np.random.normal(9, 1, n_samples) 
P = np.random.beta(1, 2, n_samples)

Y = 2*X +  np.random.normal(0, 0.5, n_samples) + P + Z + random_classes_int * X 

X= np.column_stack((X,X**2, Z + X, X * P, P, random_classes_int))

In [4]:
reg = LinearRegression(regularization="None").fit(X,Y)
ridge = LinearRegression(regularization="Ridge").fit(X,Y)
lasso = LinearRegression(regularization="Lasso", n_iter = 10000, lr=0.001).fit(X,Y)
elastic = LinearRegression(regularization="ElasticNet", n_iter = 10000, lr=0.001, alpha=0.4).fit(X,Y)

In [5]:
elastic_pred = elastic.predict(X)
elastic.print_errors(Y, elastic_pred), lasso.print_errors(Y, lasso.predict(X)), ridge.print_errors(Y, ridge.predict(X))
elastic.score(X, Y), lasso.score(X, Y), ridge.score(X, Y)

╒══════════╤═════════╕
│ Metric   │   Value │
╞══════════╪═════════╡
│ MAE      │  1.9092 │
├──────────┼─────────┤
│ RMSE     │  2.753  │
├──────────┼─────────┤
│ MSE      │  7.5791 │
╘══════════╧═════════╛
╒══════════╤═════════╕
│ Metric   │   Value │
╞══════════╪═════════╡
│ MAE      │  1.7339 │
├──────────┼─────────┤
│ RMSE     │  2.4169 │
├──────────┼─────────┤
│ MSE      │  5.8413 │
╘══════════╧═════════╛
╒══════════╤═════════╕
│ Metric   │   Value │
╞══════════╪═════════╡
│ MAE      │  1.721  │
├──────────┼─────────┤
│ RMSE     │  2.3877 │
├──────────┼─────────┤
│ MSE      │  5.7009 │
╘══════════╧═════════╛


(Array(0.87876916, dtype=float32),
 Array(0.90656686, dtype=float32),
 np.float64(0.9088114287261543))

In [6]:
#elastic.do_all(X, Y)

In [7]:
#reg.Cooks_distance(X,Y)

In [8]:
classes = ["A", "B"]
n_samples = 15

random_classes = np.random.choice(classes, size=n_samples)


class_to_int = {label: idx for idx, label in enumerate(classes)}
random_classes_int = np.vectorize(class_to_int.get)(random_classes)

X = np.random.normal(2, 2, n_samples)
Z = np.random.normal(9, 1, n_samples) 
P = np.random.beta(1, 2, n_samples)


X= np.column_stack((X, Z))

Y = random_classes_int

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1,1))

In [9]:
import pandas as pd

#X = pd.DataFrame(X, columns=["1","2","3","4","5"])

In [10]:

log = LogisticRegression(X.shape[1], 
                         len(classes), 
                         #regularization="l1", 
                         solver="GD",
                         alpha=0,
                         lr=0.0001, 
                         n_iter = 100)
log.fit(X, Y)

ValueError: Expected input batch_size (30) to match target batch_size (15).

In [ ]:
pred = log.predict(X)
pred
balanced_accuracy(Y, pred)

array(0.45535714)

In [ ]:
log.theta

array([[ 0.49572236, -0.13727251],
       [ 0.65168192,  1.51903648]])

In [ ]:
from models.onevsall import OVA
from models.svm import SVMClassificator
for i in range(0, 18):
    ova = OVA(SVMClassificator(kernel="poly", n_iter=i, lr=0.01 , C=i, r=1, degree=2 )).fit(X,Y)

    print(balanced_accuracy(Y, ova.predict(X)), i)

0.5 0
0.6428571428571428 1
0.7232142857142857 2
0.5 3
0.6517857142857143 4
0.5178571428571428 5
0.7232142857142857 6
0.5178571428571428 7
0.7232142857142857 8
0.5178571428571428 9
0.7857142857142857 10
0.5178571428571428 11
0.7857142857142857 12
0.5178571428571428 13
0.8571428571428572 14
0.5178571428571428 15
0.7946428571428572 16
0.5178571428571428 17


c:\Users\kawam\anaconda3\envs\app\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\kawam\anaconda3\envs\app\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
svm = SVMClassificator(kernel="sigmoid", n_iter=100, lr=0.01 , C=100, r=1, degree=2 ).fit(X,Y)
svm.predict(X)
print(balanced_accuracy(Y, svm.predict(X)))
svm.plot()

0.7321428571428572


IndexError: index 1 is out of bounds for axis 1 with size 1